In [1]:
import pandas as pd
import numpy as np
import json
import math

import base64
from io import BytesIO
from PIL import Image

from jinja2 import Environment, Template, FileSystemLoader

In [8]:
env = Environment()
env.filters['jsonify'] = json.dumps

In [2]:
df_numeric = pd.read_pickle('data/dataframes/pokemon_numeric.pickle')
df_names = pd.read_pickle('data/dataframes/pokemon_names.pickle')
df_sprites = pd.read_pickle('data/dataframes/pokemon_sprites.pickle.bz2', compression='bz2')
df_reduced = pd.read_pickle('data/dataframes/pokemon_pca_tsne_umap.pickle')

In [4]:
df_data = df_numeric.merge(df_names, left_index=True, right_index=True).merge(df_sprites, left_on='pokedex_number', right_on='pokedex_number').merge(df_reduced, left_index=True, right_index=True)

In [5]:
limits = {}
limits['xmin'], limits['xmax'] = (math.floor(df_data['tsne-one'].min()), math.ceil(df_data['tsne-one'].max()))
limits['ymin'], limits['ymax'] = (math.floor(df_data['tsne-one'].min()), math.ceil(df_data['tsne-one'].max()))

In [6]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f']
generations = []
for gen in df_data['generation'].unique():
    color = colors[gen - 1]
    name = 'Generation {}'.format(gen)
    series = []
    for index, row in df_data[df_data['generation'] == gen].iterrows():
        pil_img = Image.fromarray(row['image_data'])
        pil_img.thumbnail(size=(50,50))
        buff = BytesIO()
        pil_img.save(buff, format="PNG")
        image_string = base64.b64encode(buff.getvalue()).decode("utf-8")
        series.append({
            'x': row['tsne-one'],
            'y': row['tsne-two'],
            'name': row['name'],
            'image_data': image_string
        })
    generations.append({
        'name': name,
        'color': color,
        'data': series
    })

In [10]:
with open('static/highcharts_scatter_2d_template.html') as file_:
    template = file_.read()
    with open('static/highcharts_scatter_2d_gen.html', 'w') as outfile_:
        outfile_.write(env.from_string(template).render(series=generations, limits=limits))